In [ ]:
from pdf2image import convert_from_path
import os
import pandas as pd

from craft_hw_ocr import OCR
import torch
import time
from craft_text_detector import Craft
import cv2

# need to install protobuf==3.20.3
# https://github.com/protocolbuffers/protobuf/tree/main/python#installation

In [ ]:
def get_trocr_text(image_file, processor, model, craft):
    img = cv2.imread(image_file)

    # do the below step if your image is tilted by some angle else ignore
    # img = OCR.process_image(img)

    results = craft.detect_text(img)

    bboxes, text = OCR.recoginition(img, results, processor, model)

    # pilImage = OCR.visualize(img, results)

    return text

In [ ]:
models = [
"microsoft/trocr-small-printed",
"microsoft/trocr-base-printed",
"microsoft/trocr-large-printed",
"microsoft/trocr-small-handwritten",
"microsoft/trocr-base-handwritten",
"microsoft/trocr-large-handwritten",
"microsoft/trocr-small-stage1",
"microsoft/trocr-base-stage1",
"microsoft/trocr-large-str",
"microsoft/trocr-base-str",
"microsoft/trocr-large-stage1",
"DunnBC22/trocr-base-printed_captcha_ocr",
"dvsth/LEGIT-TrOCR-MT"]

In [ ]:
ocr_df = pd.read_excel('ocr_df_2023_retests.xlsx', index_col=0)

for model_name in models:
    col_name = f'trocr-{model_name}'
    ocr_df[col_name] = ''

    start_time = time.time()
    # Load Models once and pass it to the function to avoid loading models again and again
    processor, model, craft = OCR.load_models(trocr_model=model_name)
    torch.cuda.empty_cache()

    for index, row in ocr_df.iterrows():
        directory = 'test_files'

        scratch_dir = 'scratch_dir'
        for filename in os.listdir(scratch_dir):
            f = os.path.join(scratch_dir, filename)
            os.remove(f)

        pdf_text = ''
        images = convert_from_path(f'''{directory}/{row['item_filename']}.pdf''', )
        for i in range(len(images)):
            images[i].save(f'{scratch_dir}\page{str(i)}.png', 'PNG')

        for i in range(len(images)):
            generated_text = get_trocr_text(f'{scratch_dir}\page{str(i)}.png', processor, model, craft)
            pdf_text = pdf_text + "\n" + generated_text

        ocr_df.at[index, col_name] = pdf_text
        print(index, ':', time.time() - start_time)

    craft.unload_craftnet_model()
    craft.unload_refinenet_model()
    torch.cuda.empty_cache()
    print(model_name, start_time - time.time())
    ocr_df.to_excel(f'ocr_df_2023_retests_temp_3.xlsx')